# MNIST

We see FC and convolutional networks (first time) here.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

In [2]:
class FCNet(nn.Module):
    def __init__(self,din=784, dh=30, dout=10):
        super(FCNet, self).__init__()
        
        self.lin1 = nn.Linear(din, dh)
        self.lin2 = nn.Linear(dh, dout)
    
    def forward(self, x):    
        x = torch.sigmoid(self.lin1(x)) 
        x = self.lin2(x)
        return x

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
?F.nll_loss

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(-1,784)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(-1,784)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
# Training settings
input_size=(1,784,)
batch_size=64
test_batch_size=1000
epochs=10
lr=0.01
momentum=0.0   
seed=1
log_interval=100

In [12]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [13]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

In [14]:
model = FCNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [15]:
summary(model,input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]          23,550
            Linear-2                [-1, 1, 10]             310
Total params: 23,860
Trainable params: 23,860
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.09
Estimated Total Size (MB): 0.09
----------------------------------------------------------------


In [16]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.215291
Train Epoch: 1 [6400/60000 (11%)]	Loss: -2.049868
Train Epoch: 1 [12800/60000 (21%)]	Loss: -4.413392
Train Epoch: 1 [19200/60000 (32%)]	Loss: -6.953173
Train Epoch: 1 [25600/60000 (43%)]	Loss: -9.820770
Train Epoch: 1 [32000/60000 (53%)]	Loss: -12.895130
Train Epoch: 1 [38400/60000 (64%)]	Loss: -15.663527
Train Epoch: 1 [44800/60000 (75%)]	Loss: -18.660391
Train Epoch: 1 [51200/60000 (85%)]	Loss: -21.821407
Train Epoch: 1 [57600/60000 (96%)]	Loss: -24.795416

Test set: Average loss: -25.9557, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -25.997824
Train Epoch: 2 [6400/60000 (11%)]	Loss: -29.075413
Train Epoch: 2 [12800/60000 (21%)]	Loss: -32.098236
Train Epoch: 2 [19200/60000 (32%)]	Loss: -35.382168
Train Epoch: 2 [25600/60000 (43%)]	Loss: -38.304573
Train Epoch: 2 [32000/60000 (53%)]	Loss: -41.566151
Train Epoch: 2 [38400/60000 (64%)]	Loss: -44.632877
Train Epoch: 2 [44800/60000 (75%)]	Loss: -47.380630
Train Epoch: 2 [51

# Exercise

- The code will run but the network will not train. 
The problem is that FCNet output is inconsistent with the criterion used.
Modify the code in order for the FCNet to work with cross entropy.

- Then modify the code accordingly to use ConvNet (Duplicate this notebook if you find useful)